In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [3]:
mlflow.dspy.autolog()

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import dspy
import os
from dotenv import load_dotenv
from IPython.display import Image, display

load_dotenv()

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

In [5]:
response = lm("What is the capital of France?")
print(response)

['The capital of France is Paris.']


Trace(request_id=1c6987fd58bb4a2381127fae95d64a82)

In [6]:
qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")
print(response.response)

lm.inspect_history(n=1)

In Linux, "high memory" and "low memory" refer to different regions of the system's memory address space, particularly in the context of how the kernel manages memory for processes.

- **Low Memory**: This typically refers to the memory that is directly accessible by the kernel and can be used by processes without any special handling. In a 32-bit system, this is usually the first 896 MB of RAM (though this can vary based on the architecture and configuration). Low memory is used for kernel data structures and for user processes that require direct access to memory.

- **High Memory**: This refers to memory that is above the addressable range of the kernel in a 32-bit system. Since the kernel can only directly access a limited amount of memory (due to the 4 GB address space limitation), any memory above this limit is considered high memory. Processes can still use this memory, but the kernel must use special mechanisms (like paging) to access it. High memory is more relevant in systems

Trace(request_id=550a889132af46c9b8ca417da08c2b0c)

In [7]:
cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='The placement of curly braces on their own line is largely a matter of coding style and conventions. In some programming languages and style guides, such as the Allman style, it is common to place opening curly braces on a new line to enhance readability. This can help visually separate blocks of code. In contrast, other styles, like K&R style, place the opening brace on the same line as the control statement, which can save vertical space. Ultimately, whether curly braces should appear on their own line depends on the coding standards being followed in a particular project or organization.',
    response="Curly braces can appear on their own line depending on the coding style you are following. If you are using a style that emphasizes readability, such as the Allman style, then yes, they should be on their own line. If you are following a more compact style like K&R, then they would typically be on the same line. It's best to adhere to the conventions of the

Trace(request_id=010db365d4a746a19253e1e58fdbb57d)

In [8]:
import ujson
from dspy.utils import download

# Download question--answer pairs from the RAG-QA Arena "Tech" dataset.
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl")

with open("ragqa_arena_tech_examples.jsonl") as f:
    data = [ujson.loads(line) for line in f]

In [9]:
# Inspect one datapoint.
data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

In [10]:
data = [dspy.Example(**d).with_inputs('question') for d in data]

# Let's pick an `example` here from the data.
example = data[2]
example

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})

In [11]:
import random

random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

len(trainset), len(devset), len(testset)

(200, 300, 500)

In [12]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 why are my text messages coming up as maybe?

Gold Response: 	 This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". 

However, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.

Predicted Response: 	 Your text messages are showing up as "maybe" because the recipient's messaging app is unsure about the sender's identity. This can occur if you're not in their contacts or if the app is trying to filter out potential spam. To resolve this, you can ask the recipient to save your number or check their messaging app settings.

Semantic F1 Score: 0.33


[Trace(request_id=41674e0cc334491fa5804516a3af9a9f), Trace(request_id=e4489feb136c4a078a8b297f6d1cb618)]

In [13]:
# Define an evaluator that we can re-use.
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24,
                         display_progress=True, display_table=2)

# Evaluate the Chain-of-Thought program.
evaluate(cot)

Average Metric: 123.61 / 300 (41.2%): 100%|██████████| 300/300 [02:18<00:00,  2.17it/s]

2025/03/11 21:16:49 INFO dspy.evaluate.evaluate: Average Metric: 123.61108373468522 / 300 (41.2%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],"C and C++ are both powerful programming languages, but they serve ...","Use C when you need low-level system programming, performance, and...",✔️ [0.286]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",Storing images in a Git repository can be problematic for several ...,"No, images should generally not be stored in a Git repository due ...",✔️ [0.286]


41.2

[Trace(request_id=9c2d54a58a42401090e9c6149e2b1675), Trace(request_id=e43cdd1ad5be4aa5873ea044e6340d8c), Trace(request_id=2d5d8c8bdbb2468fafe2794fa67c7d3d), Trace(request_id=24644b17911e4f5f8608f9eb4c97d29c), Trace(request_id=d7bca27745f34aa2802bddf7ba73130c), Trace(request_id=8cd4080d6ce54dcf9bc41a8705aa63bb), Trace(request_id=3442f5512077413e9f6a0fa2c6ad4fa8), Trace(request_id=09af4dd6b1374730a787754f42775a34), Trace(request_id=6215161c12044018a00f80695305639b), Trace(request_id=d76ec63328904174a60849841c1d3ef4)]

In [14]:
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl")

In [16]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

with open("ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

Loaded 28436 documents. Will encode them below.
Training a 32-byte FAISS index with 337 partitions, based on 28436 x 512-dim embeddings


In [17]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [18]:
rag = RAG()
rag(question="what are high memory and low memory on linux?")

Prediction(
    reasoning="High memory and low memory in Linux refer to two distinct segments of the kernel's memory space. Low memory is the portion of memory that the kernel can access directly and is always mapped in the kernel's address space. This allows the kernel to access it without any additional steps. High memory, on the other hand, is not permanently mapped in the kernel's address space, meaning that the kernel must map it temporarily when it needs to access it. This distinction is crucial for managing memory efficiently, especially in a 32-bit architecture where the total addressable memory is limited. The kernel can access low memory directly, while high memory is used for temporary data buffers and requires specific functions (like kmap) to access.",
    response="In Linux, low memory refers to the segment of memory that the kernel can access directly and is always mapped in the kernel's address space. High memory, however, is a segment that is not permanently mapped, me

Trace(request_id=347047137f6a4af68c6b6dc8d4602ba5)

In [19]:
dspy.inspect_history()





[2025-03-11T21:40:37.919814]

System message:

Your input fields are:
1. `context` (str)
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `response` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `response`.


User message:

[[ ## context ## ]]
[1] «As far as I remember, High Memory is used for application space and Low Memory for the kernel. Advantage is that (user-space) applications cant access kernel-space memory.»
[2] «HIGHMEM is a range of kernels memory space, but it is NOT memory you access but its a place where you put what you want to access. A typical 32bit Linux virtual memory map is like: 0x00000000-0xbfffffff: user process 

In [ ]:
evaluate(RAG())